##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# MNIST classification

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/quantum/tutorials/mnist"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/quantum/blob/master/docs/tutorials/mnist.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/quantum/blob/master/docs/tutorials/mnist.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/quantum/docs/tutorials/mnist.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial builds a quantum neural network (QNN) to classify a simplified version of MNIST, similar to the approach used in <a href="https://arxiv.org/pdf/1802.06002.pdf" class="external">Farhi et al</a>. The performance of the quantum neural network on this classical data problem is compared with a classical neural network.

## Setup

In [ ]:
!pip install tensorflow==2.7.0
!pip install scikit-image
!pip install opencv-python
!pip install imutils

Install TensorFlow Quantum:

In [ ]:
!pip install tensorflow-quantum
!pip install scikit-plot

In [ ]:
# Update package resources to account for version changes.
import importlib, pkg_resources
importlib.reload(pkg_resources)
import skimage.measure
from scikitplot.metrics import plot_confusion_matrix

Now import TensorFlow and the module dependencies:

In [ ]:
import tensorflow as tf
import tensorflow_quantum as tfq

from scipy.ndimage import interpolation
import datetime
import cirq
import sympy
import numpy as np
import seaborn as sns
import collections

# visualization tools
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit
import collections
import cv2
import imutils
import matplotlib.pyplot as plt
import tensorflow as tf
from skimage.measure import block_reduce
tf.random.set_seed(10)
np.random.seed(10)
%matplotlib inline
# CONSTANTS
NUM_EXAMPLES=500
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
def plot_image(image):
  """Render image in jupyter notebook from numpy array."""
  plt.figure(figsize = (50, 15))
  plt.imshow(image)
  plt.show()

In [ ]:
# define the dictionary of digit segments so we can identify
# each digit on the thermostat
DIGITS_LOOKUP = {(1, 1, 1, 0, 1, 1, 1): 0,
                 (0, 0, 1, 0, 0, 1, 0): 1,
                 (1, 0, 1, 1, 1, 1, 0): 2,
                 (1, 0, 1, 1, 0, 1, 1): 3,
                 (0, 1, 1, 1, 0, 1, 0): 4,
                 (1, 1, 0, 1, 0, 1, 1): 5,
                 (1, 1, 0, 1, 1, 1, 1): 6,
                 (1, 0, 1, 0, 0, 1, 0): 7,
                 (1, 1, 1, 1, 1, 1, 1): 8,
                 (1, 1, 1, 1, 0, 1, 1): 9
                }

# list out keys and values separately
key_list = list(DIGITS_LOOKUP.keys())
val_list = list(DIGITS_LOOKUP.values())
x_segment_train = []
x_segment_test = []
for num in y_train:
  segment_num = np.asarray(list(key_list[val_list.index(num)]))
  x_segment_train.append(segment_num)
for num in y_test: 
  segment_num = np.asarray(list(key_list[val_list.index(num)]))
  x_segment_test.append(segment_num)


## 1. Load the data

In this tutorial you will build a binary classifier to distinguish between the digits 3 and 6, following <a href="https://arxiv.org/pdf/1802.06002.pdf" class="external">Farhi et al.</a> This section covers the data handling that:

- Loads the raw data from Keras.
- Filters the dataset to only 3s and 6s.
- Downscales the images so they fit can fit in a quantum computer.
- Removes any contradictory examples.
- Converts the binary images to Cirq circuits.
- Converts the Cirq circuits to TensorFlow Quantum circuits. 

### 1.1 Load the raw data

Load the MNIST dataset distributed with Keras. 

In [ ]:
# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# # Rescale the images from [0,255] to the [0.0,1.0] range.
x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0

# print("Number of original training examples:", len(x_train))
# print("Number of original test examples:", len(x_test))

Filter the dataset to keep just the 3s and 6s,  remove the other classes. At the same time convert the label, `y`, to boolean: `True` for `3` and `False` for 6. 

In [ ]:
# def filter_36(x, y):
#     keep = (y == 3) | (y == 6)
#     x, y = x[keep], y[keep]
#     y = y == 3
#     return x,y

In [ ]:
# x_train, y_train = filter_36(x_train, y_train)
# x_test, y_test = filter_36(x_test, y_test)

# print("Number of filtered training examples:", len(x_train))
# print("Number of filtered test examples:", len(x_test))

Show the first example:

In [ ]:
# print(y_train[0])

# plt.imshow(x_train[0, :, :, 0])
# plt.colorbar()

### 1.2 Downscale the images

An image size of 28x28 is much too large for current quantum computers. Resize the image down to 4x4:

In [ ]:
# x_train_small = tf.image.resize(x_train, (4,4)).numpy()
# x_test_small = tf.image.resize(x_test, (4,4)).numpy()

Again, display the first training example—after resize: 

In [ ]:
# print(y_train[0])

# plt.imshow(x_train_small[0,:,:,0], vmin=0, vmax=1)
# plt.colorbar()

### 1.3 Remove contradictory examples

From section *3.3 Learning to Distinguish Digits* of <a href="https://arxiv.org/pdf/1802.06002.pdf" class="external">Farhi et al.</a>, filter the dataset to remove images that are labeled as belonging to both classes.

This is not a standard machine-learning procedure, but is included in the interest of following the paper.

In [ ]:
# def remove_contradicting(xs, ys):
#     mapping = collections.defaultdict(set)
#     orig_x = {}
#     # Determine the set of labels for each unique image:
#     for x,y in zip(xs,ys):
#        orig_x[tuple(x.flatten())] = x
#        mapping[tuple(x.flatten())].add(y)
    
#     new_x = []
#     new_y = []
#     for flatten_x in mapping:
#       x = orig_x[flatten_x]
#       labels = mapping[flatten_x]
#       if len(labels) == 1:
#           new_x.append(x)
#           new_y.append(next(iter(labels)))
#       else:
#           # Throw out images that match more than one label.
#           pass
    
#     num_uniq_3 = sum(1 for value in mapping.values() if len(value) == 1 and True in value)
#     num_uniq_6 = sum(1 for value in mapping.values() if len(value) == 1 and False in value)
#     num_uniq_both = sum(1 for value in mapping.values() if len(value) == 2)

#     print("Number of unique images:", len(mapping.values()))
#     print("Number of unique 3s: ", num_uniq_3)
#     print("Number of unique 6s: ", num_uniq_6)
#     print("Number of unique contradicting labels (both 3 and 6): ", num_uniq_both)
#     print()
#     print("Initial number of images: ", len(xs))
#     print("Remaining non-contradicting unique images: ", len(new_x))
    
#     return np.array(new_x), np.array(new_y)

The resulting counts do not closely match the reported values, but the exact procedure is not specified.

It is also worth noting here that applying filtering contradictory examples at this point does not totally prevent the model from receiving contradictory training examples: the next step binarizes the data which will cause more collisions. 

In [ ]:
#x_train_nocon, y_train_nocon = remove_contradicting(x_train_small, y_train)

### 1.4 Encode the data as quantum circuits

To process images using a quantum computer, <a href="https://arxiv.org/pdf/1802.06002.pdf" class="external">Farhi et al.</a> proposed representing each pixel with a qubit, with the state depending on the value of the pixel. The first step is to convert to a binary encoding.

In [ ]:
THRESHOLD = 0.5

# x_train_bin = np.array(x_train_nocon > THRESHOLD, dtype=np.float32)
# x_test_bin = np.array(x_test_small > THRESHOLD, dtype=np.float32)

If you were to remove contradictory images at this point you would be left with only 193, likely not enough for effective training.

In [ ]:
def moments(image):
    c0,c1 = np.mgrid[:image.shape[0],:image.shape[1]] # A trick in numPy to create a mesh grid
    totalImage = np.sum(image) #sum of pixels
    m0 = np.sum(c0*image)/totalImage #mu_x
    m1 = np.sum(c1*image)/totalImage #mu_y
    m00 = np.sum((c0-m0)**2*image)/totalImage #var(x)
    m11 = np.sum((c1-m1)**2*image)/totalImage #var(y)
    m01 = np.sum((c0-m0)*(c1-m1)*image)/totalImage #covariance(x,y)
    mu_vector = np.array([m0,m1]) # Notice that these are \mu_x, \mu_y respectively
    covariance_matrix = np.array([[m00,m01],[m01,m11]]) # Do you see a similarity between the covariance matrix
    return mu_vector, covariance_matrix
#Deskew the training samples 
def deskew(image):
    c,v = moments(image)
    alpha = v[0,1]/v[0,0]
    affine = np.array([[1,0],[alpha,1]])
    ocenter = np.array(image.shape)/2.0
    offset = c-np.dot(affine,ocenter)
    img = interpolation.affine_transform(image,affine,offset=offset)
    return (img - img.min()) / (img.max() - img.min())


The qubits at pixel indices with values that exceed a threshold, are rotated through an $X$ gate.

In [ ]:
#training set 
x_train_deskew = [] 
for i in range(x_train.shape[0]): 
    x_train_deskew.append(deskew(x_train[i].reshape(28,28)))
x_train_deskew = np.array(x_train_deskew)
x_train_deskew = x_train_deskew[..., np.newaxis]
print("shape of x_train_deskew is " + str(np.shape(x_train_deskew)))
print("type of x_train_deskew is " + str(type(x_train_deskew)))

#test set 
x_test_deskew = [] 
for j in range(x_test.shape[0]): 
    x_test_deskew.append(deskew(x_test[j].reshape(28,28)))
x_test_deskew = np.array(x_test_deskew)
x_test_deskew = x_test_deskew[..., np.newaxis]
print("shape of x_test_deskew is " + str(np.shape(x_test_deskew)))
print("type of x_test_deskew is " + str(type(x_test_deskew)))


In [ ]:
# Multi Average Pooling

In [ ]:
plt.imshow(x_test[1].reshape(28,28))

In [ ]:
test_img=x_test_deskew[10].reshape(28,28)[5:25,5:25]
plt.imshow(x_test_deskew[10].reshape(28,28)[5:25,5:25])

In [ ]:
kernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])
image_sharp = cv2.filter2D(src=test_img, ddepth=-1, kernel=kernel)

In [ ]:
new_image= skimage.measure.block_reduce(test_img, (2,2), np.mean)

In [ ]:
plt.imshow(new_image)
len(new_image)

In [ ]:
THRESHOLD = 0.0
test_thresh = np.array(new_image > THRESHOLD, dtype=np.float32)

In [ ]:
plt.imshow(test_thresh)

In [ ]:
test_2 = skimage.measure.block_reduce(test_thresh, (1,2), np.mean)

In [ ]:
test_thresh[1]
x = test_thresh[1]/np.linalg.norm(test_thresh[1])
theta = test_thresh[1][0:3]/np.linalg.norm(test_thresh[1][0:3])
phi = test_thresh[1][3:6]/np.linalg.norm(test_thresh[1][3:6])
psi = test_thresh[1][6:9]/np.linalg.norm(x)
print(theta, phi, psi)

In [ ]:
def sectioned_angle_embedding(img, sections=3):
  section_angles = []
  for row in img:
    section = int(np.floor(len(row)/sections))
    prior = None
    angles = []
    for i in range(sections):
      if prior is None:
        prior = 0
      angle = np.mean(row[prior:prior+section])/np.pi
      angles.append(angle)
      prior += section
    section_angles.append(angles)
  return section_angles


      #we want to get the angle values for each section between 0. and p
x = sectioned_angle_embedding(test_thresh)
print(x)

In [ ]:
test_2 = skimage.measure.block_reduce(test_thresh, (2,2), np.max)
plt.imshow(test_2)

In [ ]:
test_3 = skimage.measure.block_reduce(test_2, (2,1), np.mean)
plt.imshow(test_3)

In [ ]:

new_image_sharpend = cv2.filter2D(src=test_2, ddepth=-5, kernel=kernel)
plt.imshow(new_image_sharpend)

In [ ]:
THRESHOLD2  = 0.4
test_final = np.array(test_3> THRESHOLD2, dtype=np.float32)
plt.imshow(test_final)

In [ ]:
resized = cv2.resize(test_thresh,(4,4))
plt.imshow(resized)

In [ ]:
# CROP, AVERAGE POOL THEN THRESHOLD
THRESHOLD =0.3
x_train_avg = np.array([skimage.measure.block_reduce(x.reshape(28,28)[5:25,5:25], (2,2), np.mean) for x in x_train_deskew])
x_test_avg =np.array([skimage.measure.block_reduce(x.reshape(28,28)[5:25,5:25], (2,2), np.mean) for x in x_test_deskew])

x_train_deskew_bin = np.array(x_train_avg > THRESHOLD, dtype=np.float32)
x_test_deskew_bin = np.array(x_test_avg > THRESHOLD, dtype=np.float32)
# x_train_deskew_bin = np.array([skimage.measure.block_reduce(x.reshape(28,28)[5:25,5:25], (2,2), np.mean) for x in x_train_deskew])
# x_test_deskew_bin  =np.array([skimage.measure.block_reduce(x.reshape(28,28)[5:25,5:25], (2,2), np.mean) for x in x_test_deskew])

In [ ]:
x = 3336
plt.imshow(x_train_deskew_bin[0])
print(y_train[x])

In [ ]:
# def convert_to_circuit(image):
#     """Encode truncated classical image into quantum datapoint."""
#     values = np.ndarray.flatten(image)
#     qubits = cirq.LineQubit.range(7)
#     circuit = cirq.Circuit()
#     for i, value in enumerate(values):
#         if value:
#             circuit.append(cirq.X(qubits[i]))
#     return circuit

# x_train_circ = [convert_to_circuit(x) for x in x_train_deskew]
# x_test_circ = [convert_to_circuit(x) for x in x_test_deskew]

In [ ]:
def section_angle_embedding_circ(img):
  angle_list = sectioned_angle_embedding(img)
  qubits = cirq.LineQubit.range(10)
  circuit = cirq.Circuit()
  for i, value in enumerate(angle_list):
    r_x,r_y,r_z,  = value
    circuit.append(cirq.X(qubits[i])**(r_x))
    circuit.append(cirq.Y(qubits[i])**(r_y))
    circuit.append(cirq.Z(qubits[i])**(r_z))
  return circuit

x_train_circ = [section_angle_embedding_circ(x) for x in x_train_deskew_bin]
x_test_circ = [section_angle_embedding_circ(x) for x in x_test_deskew_bin]

In [ ]:
section_angle_embedding_circ(x_train_avg[0])

Here is the circuit created for the first example (circuit diagrams do not show qubits with zero gates):

In [ ]:
SVGCircuit(x_train_circ[0])

Compare this circuit to the indices where the image value exceeds the threshold:

Convert these `Cirq` circuits to tensors for `tfq`:

In [ ]:
x_train_tfcirc = tfq.convert_to_tensor(x_train_circ)
x_test_tfcirc = tfq.convert_to_tensor(x_test_circ)


## 2. Quantum neural network

There is little guidance for a quantum circuit structure that classifies images. Since the classification is based on the expectation of the readout qubit, <a href="https://arxiv.org/pdf/1802.06002.pdf" class="external">Farhi et al.</a> propose using two qubit gates, with the readout qubit always acted upon. This is similar in some ways to running small a <a href="https://arxiv.org/abs/1511.06464" class="external">Unitary RNN</a> across the pixels.

In [ ]:
print(len(x_train_tfcirc),len(x_test_tfcirc))

### 2.1 Build the model circuit

This following example shows this layered approach. Each layer uses *n* instances of the same gate, with each of the data qubits acting on the readout qubit.

Start with a simple class that will add a layer of these gates to a circuit:

In [ ]:
class CircuitLayerBuilder():
    def __init__(self, data_qubits, readout):
        self.data_qubits = data_qubits
        self.readout = readout
    
    def add_layer(self, circuit, single_qubit_gate,two_qubit_gate, prefix):
        for i, qubit in enumerate(self.data_qubits):
          if i < len(self.data_qubits)-1:
            symbol = sympy.Symbol(prefix + '-' + str(i))
            circuit.append(two_qubit_gate(qubit, self.data_qubits[i+1]))
            circuit.append(single_qubit_gate(self.data_qubits[i+1])**(2*symbol))
            circuit.append(two_qubit_gate(qubit, self.data_qubits[i+1]))

In [ ]:
cirq.LineQubit(0)

Build an example circuit layer to see how it looks:

In [ ]:
demo_builder = CircuitLayerBuilder(data_qubits = cirq.LineQubit.range(7),
                                   readout=cirq.LineQubit(-1))
circuit = cirq.Circuit()
demo_builder.add_layer(circuit, cirq.Y, cirq.CNOT, prefix='xx')
SVGCircuit(circuit)

Now build a two-layered model, matching the data-circuit size, and include the preparation and readout operations.

In [ ]:
def create_quantum_model():
    """Create a QNN model circuit and readout operation to go along with it."""
    data_qubits =  cirq.LineQubit.range(10)  # a 4x4 grid.
    readout =cirq.LineQubit.range(10)     # a single qubit at [-1,-1]
    circuit = cirq.Circuit()
    
    builder = CircuitLayerBuilder(
        data_qubits = data_qubits,
        readout=readout)

    # # Then add layers (experiment by adding more).
    builder.add_layer(circuit,cirq.Y, cirq.CNOT, "cnot1")
    builder.add_layer(circuit, cirq.X, cirq.CNOT, "cnot2")
    builder.add_layer(circuit, cirq.Z, cirq.CNOT, "cnot3")

    # Finally, prepare the readout qubit.
    measurements = [cirq.X, cirq.Y,cirq.Z]
    rng = np.random.default_rng(12345)
    rints = rng.integers(low=0, high=2, size=1)[0]
    readout_measures = []
    for i, qubit in enumerate(readout):
      for gate in measurements:
        readout_measures.append(gate(qubit))

    return circuit, readout_measures

In [ ]:
model_circuit, model_readout = create_quantum_model()

In [ ]:
depth = len(cirq.Circuit(model_circuit.all_operations()))
depth

### 2.2 Wrap the model-circuit in a tfq-keras model

Build the Keras model with the quantum components. This model is fed the "quantum data", from `x_train_circ`, that encodes the classical data. It uses a *Parametrized Quantum Circuit* layer, `tfq.layers.PQC`, to train the model circuit, on the quantum data.

To classify these images, <a href="https://arxiv.org/pdf/1802.06002.pdf" class="external">Farhi et al.</a> proposed taking the expectation of a readout qubit in a parameterized circuit. The expectation returns a value between 1 and -1.

In [ ]:
SVGCircuit(model_circuit)

In [ ]:
# Build the Keras model.
model = tf.keras.Sequential([
    # The input is the data-circuit, encoded as a tf.string
    tf.keras.layers.Input(shape=(), dtype=tf.string),
    # The PQC layer returns the expected value of the readout gate, range [-1,1].
    tfq.layers.PQC(model_circuit, model_readout),
    tf.keras.layers.Dense(10, activation='softmax')
])

Next, describe the training procedure to the model, using the `compile` method.

Since the the expected readout is in the range `[-1,1]`, optimizing the hinge loss is a somewhat natural fit. 

Note: Another valid approach would be to shift the output range to `[0,1]`, and treat it as the probability the model assigns to class `3`. This could be used with a standard a `tf.losses.BinaryCrossentropy` loss.

To use the hinge loss here you need to make two small adjustments. First convert the labels, `y_train_nocon`, from boolean to `[-1,1]`, as expected by the hinge loss.

In [ ]:
y_train_onehot = tf.one_hot(y_train,10)
y_test_onehot = tf.one_hot(y_test,10)

Second, use a custiom `hinge_accuracy` metric that correctly handles `[-1, 1]` as the `y_true` labels argument. 
`tf.losses.BinaryAccuracy(threshold=0.0)` expects `y_true` to be a boolean, and so can't be used with hinge loss).

In [ ]:
def hinge_accuracy(y_true, y_pred):
    y_true = tf.squeeze(y_true) > 0.0
    y_pred = tf.squeeze(y_pred) > 0.0
    result = tf.cast(y_true == y_pred, tf.float32)

    return tf.reduce_mean(result)

In [ ]:
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
    optimizer=tf.keras.optimizers.Adam(0.02))

In [ ]:
print(model.summary())

### Train the quantum model

Now train the model—this takes about 45 min. If you don't want to wait that long, use a small subset of the data (set `NUM_EXAMPLES=500`, below). This doesn't really affect the model's progress during training (it only has 32 parameters, and doesn't need much data to constrain these). Using fewer examples just ends training earlier (5min), but runs long enough to show that it is making progress in the validation logs.

In [ ]:
EPOCHS = 5
BATCH_SIZE = 32

NUM_EXAMPLES = 500

In [ ]:
x_train_tfcirc_sub = x_train_tfcirc[:NUM_EXAMPLES]
y_train_sub = y_train[:NUM_EXAMPLES]


In [ ]:
# CALLBACKS
# Load the TensorBoard notebook extension
%load_ext tensorboard
current_time = str(datetime.datetime.now().timestamp())
train_log_dir = 'logs/tensorboard/' + current_time
test_log_dir = 'logs/tensorboard/test/' + current_time
path_for_checkpoint_callback = 'logs/summary/'+current_time

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss',
                              patience=7, min_delta=1e-7, verbose=1)
tensorboard = tf.keras.callbacks.TensorBoard(
                log_dir=train_log_dir, histogram_freq=1, profile_batch=3
            )
model_ckpt =  tf.keras.callbacks.ModelCheckpoint(
                path_for_checkpoint_callback, save_weights_only=True
            )

Training this model to convergence should achieve >85% accuracy on the test set.

In [ ]:
qnn_history = model.fit(x_train_tfcirc,
          y_train_onehot,
          batch_size=32,
          epochs=125,
          verbose=1,
          callbacks=[tensorboard,reduce_lr,model_ckpt]) 
qnn_results = model.evaluate(x_test_tfcirc, y_test_onehot)


In [ ]:
print("Weights and biases of the layers before training the model: \n")
for layer in model.layers:
  print(layer.name)
  print("Weights")
  print("Shape: ",layer.get_weights()[0].shape,'\n',layer.get_weights()[0])

In [ ]:
 qnn_results = model.evaluate(x_test_tfcirc, y_test_onehot)

In [ ]:
y_pred = model.predict(x_test_tfcirc)
y_pred = np.argmax(y_pred, axis=1)
print(y_pred, y_test)


In [ ]:
fig, ax = plt.subplots(figsize=(16,12))
plot_confusion_matrix(y_test, y_pred, ax=ax)

Note: The training accuracy reports the average over the epoch. The validation accuracy is evaluated at the end of each epoch.

## 3. Classical neural network

While the quantum neural network works for this simplified MNIST problem, a basic classical neural network can easily outperform a QNN on this task. After a single epoch, a classical neural network can achieve >98% accuracy on the holdout set.

In the following example, a classical neural network is used for for the 3-6 classification problem using the entire 28x28 image instead of subsampling the image. This easily converges to nearly 100% accuracy of the test set.

In [ ]:

def create_classical_model():
    # A simple model based off LeNet from https://keras.io/examples/mnist_cnn/
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(32, [3, 3], activation='relu', input_shape=(10,10,1)))
    model.add(tf.keras.layers.Conv2D(64, [3, 3], activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    return model


model = create_classical_model()
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(0.002),
               metrics=[tf.keras.metrics.CategoricalAccuracy()])

model.summary()

In [ ]:
model.fit(x_train_deskew_bin,
          y_train_onehot,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test_deskew_bin[:1000], y_test_onehot[:1000]))

cnn_results = model.evaluate(x_test_deskew_bin, y_test_onehot)

The above model has nearly 1.2M parameters. For a more fair comparison, try a 37-parameter model, on the subsampled images:

In [ ]:
def create_fair_classical_model():
    # A simple model based off LeNet from https://keras.io/examples/mnist_cnn/
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(10,10)))
    model.add(tf.keras.layers.Dense(3))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    return model


model = create_fair_classical_model()
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(0.002),
              metrics=['accuracy'])
model.summary()

In [ ]:

model.fit(x_train_deskew_bin,
          y_train_onehot,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(x_test_deskew_bin[:1000], y_test_onehot[:1000]))

fair_nn_results = model.evaluate(x_test_deskew_bin , y_test_onehot)

## 4. Comparison

Higher resolution input and a more powerful model make this problem easy for the CNN. While a classical model of similar power (~32 parameters) trains to a similar accuracy in a fraction of the time. One way or the other, the classical neural network easily outperforms the quantum neural network. For classical data, it is difficult to beat a classical neural network.

In [ ]:
qnn_accuracy = qnn_results[1]
cnn_accuracy = cnn_results[1]
fair_nn_accuracy = fair_nn_results[1]

sns.barplot(["Quantum", "Classical, full", "Classical, fair"],
            [qnn_accuracy, cnn_accuracy, fair_nn_accuracy])

In [ ]:
qnn_results[1]